In [ ]:
import os
import json
import re
from dotenv import load_dotenv
from flask import Flask, request, session, redirect, url_for, render_template
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_groq import ChatGroq
from deep_translator import GoogleTranslator

load_dotenv()


app = Flask(__name__)
app.secret_key = os.getenv("FLASK_SECRET_KEY", "default_secret")
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"

google_tool = DuckDuckGoSearchRun(name="Google Search")
wikipedia_tool_en = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(lang="en", top_k_results=3),
    description="Fetch information from Wikipedia in English."
)
wikipedia_tool_ar = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(lang="ar", top_k_results=3),
    description="الحصول على معلومات"
)

def clean_question(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    return text


class AISystem:
    def __init__(self):
        self.llm = ChatGroq(
            temperature=0,
            model_name="llama3-70b-8192",
            max_tokens=4096,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )

        self.prompt_ar = ChatPromptTemplate.from_messages([
            ("system", "أنت مساعد طبي ذكي. أجب على الأسئلة الطبية باللغة العربية باستخدام ويكيبيديا أو جوجل بدقة."),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
            ("human", "{input}")
        ])

        self.prompt_en = ChatPromptTemplate.from_messages([
            ("system", "You are a smart medical assistant. Answer medical questions in English using Wikipedia or Google."),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
            ("human", "{input}")
        ])

        self.agent_ar = create_tool_calling_agent(
            llm=self.llm,
            tools=[google_tool, wikipedia_tool_ar],
            prompt=self.prompt_ar
        )

        self.agent_en = create_tool_calling_agent(
            llm=self.llm,
            tools=[google_tool, wikipedia_tool_ar],
            prompt=self.prompt_en
        )

        self.executor_ar = AgentExecutor(agent=self.agent_ar, tools=[google_tool, wikipedia_tool_ar], verbose=True)
        self.executor_en = AgentExecutor(agent=self.agent_en, tools=[google_tool, wikipedia_tool_ar], verbose=True)

       
        self.cache_path = "cache.json"
        if os.path.exists(self.cache_path):
            with open(self.cache_path, "r", encoding="utf-8") as f:
                self.cache = json.load(f)
        else:
            self.cache = {}

        with open("questions_ar.json", "r", encoding="utf-8") as f:
            self.questions_ar = json.load(f)

      

    def save_cache(self):
        with open(self.cache_path, "w", encoding="utf-8") as f:
            json.dump(self.cache, f, ensure_ascii=False, indent=2)

    def get_sperm_info(self, question: str, is_arabic: bool) -> str:
        q = clean_question(question)

        
        for k, v in self.questions_ar.items():
            if clean_question(k) in q:
                return v

        
        if q in self.cache:
            return self.cache[q]

       
        try:
            if is_arabic:
                result = self.executor_ar.invoke({"input": question})["output"]
            else:
                result = self.executor_en.invoke({"input": question})["output"]

            if is_arabic and not all('\u0600' <= c <= '\u06FF' for c in result):
                result = GoogleTranslator(source="auto", target="ar").translate(result)

            
            self.cache[q] = result
            self.save_cache()
            return result

        except Exception as e:
            return f"حدث خطأ أثناء جلب المعلومات: {str(e)}"

    def get_response(self, question: str) -> str:
        is_arabic = any('\u0600' <= c <= '\u06FF' for c in question)
        clean_q = clean_question(question)

        
        if clean_q in self.cache:
            return self.cache[clean_q]

        
        sperm_keywords = ['sperm', 'الحيوانات المنويه', 'حيوان منوي', 'الحيوانات المنوية']

        if any(keyword in clean_q for keyword in sperm_keywords):
            sperm_answer = self.get_sperm_info(question, is_arabic)
            self.cache[clean_q] = sperm_answer
            self.save_cache()
            return sperm_answer

        try:
            
            if is_arabic:
                result = self.executor_ar.invoke({"input": question})["output"]
            else:
                result = self.executor_en.invoke({"input": question})["output"]

            if is_arabic and not all('\u0600' <= c <= '\u06FF' for c in result):
                result = GoogleTranslator(source="auto", target="ar").translate(result)

            self.cache[clean_q] = result
            self.save_cache()
            return result
        except Exception as e:
            return f"حدث خطأ أثناء المعالجة: {str(e)}"


ai_system = AISystem()

@app.route('/', methods=['GET', 'POST'])
def index():
    if 'history' not in session:
        session['history'] = []

    if request.method == 'POST':
        question = request.form.get('question', '').strip()
        if question:
            session['history'].append(('user', question))
            response = ai_system.get_response(question)
            session['history'].append(('bot', response))

            if len(session['history']) > 4:
                session['history'] = session['history'][-4:]

            session.modified = True
        return redirect(url_for('index'))

    return render_template('index.html', history=session.get('history', []))

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Jun/2025 22:31:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2025 22:31:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2025 22:32:05] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [10/Jun/2025 22:32:05] "GET / HTTP/1.1" 200 -
